# spotter

This is a text cell. Start editing!

In [1]:
val schemaRegistryURL = "http://schema-registry:8081"
val topic = "spot-price-topic"

In [2]:
val df = spark.read
        .format("kafka")
        .option("kafka.bootstrap.servers", "broker:9092")
        .option("subscribe", topic)
        .option("startingOffsets", "earliest") // From starting
        .load()
//df.printSchema

In [3]:
import io.confluent.kafka.schemaregistry.client.CachedSchemaRegistryClient

val schemaRegistryClient = new CachedSchemaRegistryClient(schemaRegistryURL, 128)

val valueSchema = schemaRegistryClient.getLatestSchemaMetadata(topic + "-value").getSchema

In [4]:
import org.apache.spark.sql.avro._
import org.apache.spark.sql.types.DataTypes
val avroDf = df.select(
  'key.cast(DataTypes.StringType),
  from_avro('value, valueSchema).as("value")
)

In [5]:
avroDf.printSchema

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- provider: string (nullable = false)
 |    |-- zone: string (nullable = false)
 |    |-- instance: string (nullable = false)
 |    |-- timestamp: timestamp (nullable = false)
 |    |-- price: double (nullable = false)



In [6]:
avroDf.select("value.*", "value.timestamp")

[provider: string, zone: string ... 4 more fields]